# Twitter REST API for fun and profit

[Twitter REST API reference](https://dev.twitter.com/rest/reference)

[Twitter Search API reference](https://dev.twitter.com/rest/public/search)

## Boilerplate

In [1]:
import statistics

import requests

consumer_key = '96szQi0MOlGVJV0A01krEX1t6'
consumer_secret = 'Pp8F7vuqxVycEk8zkleaod0RqZyLGYQQEb9FBJWRflY27LyYlB'

app_auth_url = 'https://api.twitter.com/oauth2/token'
app_auth_url
auth_resp = requests.post(
    app_auth_url,
    auth=(consumer_key, consumer_secret),
    data={'grant_type': 'client_credentials'}
)

auth_resp_data = auth_resp.json()
bearer_token = auth_resp_data['access_token']
bearer = 'Bearer ' + bearer_token

HEADERS = {'Authorization': bearer}
API_ROOT = 'https://api.twitter.com/1.1/'

In [2]:
bearer

'Bearer AAAAAAAAAAAAAAAAAAAAAMWXxwAAAAAA6yrL1ByLXpPvyrXturxZWvJ%2FDrc%3DwzTqOQafZU1kERLOIiLrNmSvoRxGz5BmkdzfxYgERZwA0ZwQJo'

# User timelines

In [3]:
url = API_ROOT + 'statuses/user_timeline.json'

params = {
    'screen_name': 'potus',
}

resp = requests.get(url, headers=HEADERS, params=params)
user_timeline = resp.json()

In [4]:
type(user_timeline)

len(user_timeline)

20

In [5]:
user_timeline[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Jul 24 20:18:34 +0000 2017',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'pscp.tv/w/bET7MDEyMzE3…',
    'expanded_url': 'https://www.pscp.tv/w/bET7MDEyMzE3NDF8MXlwS2Rsek9rT0xLV_x3_XnsKc_UJsJyrpNIioPSuUq_gF8aFXYM8CruZ4vf',
    'indices': [112, 135],
    'url': 'https://t.co/Yxm94XKtuv'}],
  'user_mentions': [{'id': 25073877,
    'id_str': '25073877',
    'indices': [3, 19],
    'name': 'Donald J. Trump',
    'screen_name': 'realDonaldTrump'},
   {'id': 830128820447539208,
    'id_str': '830128820447539208',
    'indices': [57, 72],
    'name': 'Rick Perry',
    'screen_name': 'SecretaryPerry'},
   {'id': 827258161841135623,
    'id_str': '827258161841135623',
    'indices': [74, 89],
    'name': 'Secretary Ryan Zinke',
    'screen_name': 'SecretaryZinke'},
   {'id': 829782369670410240,
    'id_str': '829782369670410240',
    'indices': [95, 106],
    'name': 'Tom Price, M.D.',
    'screen_name

In [6]:
resp.headers['x-rate-limit-remaining']

'1499'

In [7]:
url = API_ROOT + 'statuses/user_timeline.json'



params = {
    'screen_name': 'potus',
    'max_id': min(t['id'] for t in user_timeline) -1
}

resp = requests.get(url, headers=HEADERS, params=params)
user_timeline += resp.json()

In [8]:
len(user_timeline)

40

In [9]:
resp.headers['x-rate-limit-remaining']

'1498'

# Verify friendship paradox

In [10]:
url = API_ROOT + 'users/show.json'
params = {
    'screen_name': 'clayadavis',
}
resp = requests.get(url, headers=HEADERS, params=params)
user = resp.json()

In [11]:
user['friends_count']

166

In [12]:
url = API_ROOT + 'friends/ids.json'
params = {
    'screen_name': 'clayadavis',
}
resp = requests.get(url, headers=HEADERS, params=params)
friends_ids = resp.json()['ids']
len(friends_ids)

166

In [13]:
url = API_ROOT + 'users/lookup.json'
params = {
    'user_id': friends_ids[:100],
}
resp = requests.get(url, headers=HEADERS, params=params)
friends = resp.json()
#friends[0]

In [14]:
friends_count = [u['friends_count'] for u in friends]
print (friends_count)
statistics.mean(friends_count)

[20, 3, 904, 952, 107, 354, 365, 240, 18, 597, 2425, 269, 28, 196, 390, 125, 95, 42, 2723, 365, 608, 804, 19, 600, 421, 125, 3119, 525, 2290, 316, 1196, 54, 2608, 614, 1689, 308, 35, 239, 2762, 1515, 96, 282, 234, 8189, 1016, 178, 39, 29, 151, 106, 490, 60, 627, 1397, 261, 18, 2794, 233, 429, 1326, 403, 1669, 598, 297, 1528, 2018, 3, 734, 89, 9699, 8, 543, 765, 406, 514, 2340, 717, 305, 1705, 685, 209, 169, 120, 8, 39, 0, 1328, 600, 870, 76, 661, 673, 1286, 644, 148, 432, 913, 79, 190, 6]


814.97

In [15]:
statistics.median(friends_count)

396.5

# Search

In [16]:
url = API_ROOT + 'search/tweets.json'
params = {
    'q': '#wud2016',
    'result_type': 'recent',
    'count': 100,
}
resp = requests.get(url, headers=HEADERS, params=params)
search_data = resp.json()

In [24]:
statuses = search_data['statuses']

In [18]:
import networkx as nx

retweet_graph = nx.DiGraph()
for status in statuses:
    if 'retweeted_status' in status:
        retweeted_status = status['retweeted_status']
        target = status['user']['screen_name']
        source = retweeted_status['user']['screen_name']
        if retweet_graph.has_edge(source, target):
            retweet_graph.edge[source][target]['tweet_ids']\
            .append(status['id'])
        else:
            retweet_graph.add_edge(source, target, 
                                   tweet_ids=[status['id']])

In [19]:
nx.draw(retweet_graph)

In [20]:
edge_widths = []
for source, target in retweet_graph.edges_iter():
    tweet_ids = retweet_graph.edge[source][target]['tweet_ids']
    edge_widths.append(len(tweet_ids))

In [21]:
nx.draw(retweet_graph, width=edge_widths)